# D-DUST Data Grid Processing Notebook

- - -

Check the following link for Data Management Plan and the Variable list table: <br>
**1. [Data Management Plan (DMP)](https://docs.google.com/document/d/1n3PVat7PBTG76JnINOkL2pvBZuKQlakZkTgqNj39oAQ/edit#)**<br>
**2. [Variables list table](https://docs.google.com/spreadsheets/d/1-5pwMSc1QlFyC8iIaA-l1fWhWtpqVio2/edit#gid=91313358)**

This notebook describes the physical variables selected for the project and how they are preprocessed.
These variables are divided into <u>4 categories</u>, as shown in the Variable list table:
1. **Map Layer**: static layer used to describe Lombardy region morphology and its features (such as elevation, infrastructures, land use and cover etc.)
2. **Model**: data retrieved from a model that uses satellite and in-situ observations of meteorological and air quality data as input (such as ERA5, CAMS).
3. **Satellite**: data obtained directly from satellite observations (such as Sentinel-5P).
4. **Ground Sensor**: data retrieved from ground monitoring stations measuring air quality and meteorological variables.

First the other notebooks must be used in order to retrieve all the data in the same time range.
- GEE_API Notebook: satellite images + ERA5 data
- ADS_API Copernicus Notebook: ADS data from CAMS analysis model
- ARPA_API Ground Sensors Notebook: data from ARPA ground sensor
- AQ_ESA Stations Notebook: data from ESA Air Quality station


## Import libraries

In [1]:
import os
import pandas as pd
import geopandas as gpd
import numpy as np
import rasterio as rio
import scipy.interpolate
from scipy.interpolate import griddata
import rasterstats as rstat
import rioxarray
import shapely.speedups
shapely.speedups.enable()
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
from shapely.geometry import shape
from shapely.geometry import  MultiLineString
from rasterio.enums import Resampling
import ipywidgets as widgets

import warnings
warnings.filterwarnings('ignore')

In [2]:
from functions import select_grid as sg

In [3]:
cwd = os.getcwd()

## Import grids

Three grids with different spatial resolution are used in this project:
1. **cams** grid: 0.1° x 0.1° resolution - Grid with CAMS Model spatial resolution.
2. **s5p** grid: 0.066° x 0.066° resolution - Grid with the Sentinel-5P approximate spatial resolution.
3. **arpa** grid: 0.01° x 0.01° resolution- Grid generated with at most one ARPA monitoring station for each pixel.

These grids are defined as bounding box of the Lombardy region layer applying a buffer of 20 km.

In [4]:
# With this widget is possible to select from the dropdown list
name = widgets.Dropdown(
    options=['cams', 's5p', 'arpa'],
    description='name:',
    disabled=False)
name

Dropdown(description='name:', options=('cams', 's5p', 'arpa'), value='cams')

In [5]:
#to interpolate CAMS .tif file with a lower resolution depending on the grid cell size (useful for zonal statistics)
if name.value == 'cams':
    upscale_factor = 1  
if name.value == 's5p':
    upscale_factor = 10
if name.value == 'arpa':
    upscale_factor = 10  

grid_path = sg.select_grid(name.value)

In [6]:
grid = gpd.read_file(cwd + grid_path)

In [7]:
m_to_km = 10**(-3)
m2_to_km2 = 10**(-6)

---

# Importing Map Layers

### [DUSAF - Land use - Geoportale Lombardia](https://www.geoportale.regione.lombardia.it/metadati?p_p_id=detailSheetMetadata_WAR_gptmetadataportlet&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&_detailSheetMetadata_WAR_gptmetadataportlet_uuid=%7B18EE7CDC-E51B-4DFB-99F8-3CF416FC3C70%7D) <br>

Consists in a multi-temporal geographic database that classifies land based on major land cover and land use types. Reference system EPSG:4326.<br>
Land use:
- 2 = Aree agricole.
- 3 =Territori boscati e ambienti seminaturali.
- 4 = Aree umide.
- 5 = Corpi idrici.
- 11 = Zone urbanizzate.
- 12 = Insediamenti produttivo, grandi impianti e reti di comunicazione.
- 13 = Aree estrattive, discariche, cantieri, terreni artefatti e abbandonati.
- 14 = Aree verdi non agricole.

In [8]:
dusaf_path = cwd + '/land_use_cover/DUSAF6_dissolve_rast_4326.tiff'

In [9]:
dusaf = rio.open(dusaf_path)
dusaf_array = dusaf.read(1).astype('float64') 
dusaf_array[dusaf_array<1.0]=np.nan
affine = dusaf.transform

In [10]:
# Class majority calculation for each cell
stats = rstat.zonal_stats(grid, dusaf_array, affine=affine, nodata=np.nan, stats=['majority'], categorical=True)
majority_list = [{k: v for k, v in d.items() if k == 'majority'} for d in stats]
grid = grid.join(pd.DataFrame(majority_list), how='left')
grid = grid.rename(columns={"majority": "dusaf"})

In [11]:
# Class counts in each tile
stats = rstat.zonal_stats(grid, dusaf_array, affine=affine, nodata=np.nan, stats=['count'], categorical=True)
p = pd.DataFrame.from_dict(stats, orient='columns')
p = p*m2_to_km2

grid['dsf2'] = p[2.0]
grid['dsf3'] = p[3.0]
grid['dsf4'] = p[4.0]
grid['dsf5'] = p[5.0]
grid['dsf11'] = p[11.0]
grid['dsf12'] = p[12.0]
grid['dsf13'] = p[13.0]
grid['dsf14'] = p[14.0]
grid['dsfSum'] = p['count']

 - - -

### [SIARL - Agricultural use - Geoportale Lombardia](https://www.geoportale.regione.lombardia.it/metadati?p_p_id=detailSheetMetadata_WAR_gptmetadataportlet&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&_detailSheetMetadata_WAR_gptmetadataportlet_uuid=%7B83483117-8742-4A1F-A16E-3A48AEE2EBE2%7D) <br>
This layer contains the agricoltural use for each cadastral parcel provided by SIARL 2019 Catalog for the Lombardy region. Reference system EPSG:4326. <br>
Agricoltural use:
- 2 Altri cereali
- 7 Foraggere
- 9 Mais
- 12 Riso

In [12]:
siarl_path = cwd + '/land_use_cover/siarl.tif'
siarl = rio.open(siarl_path)

In [13]:
siarl_array = siarl.read(1).astype('float64') 
siarl_array[siarl_array<1.0]=np.nan
affine = siarl.transform

In [14]:
# Class majority calculation for each cell
stats_siarl = rstat.zonal_stats(grid, siarl_array, affine=affine, nodata=np.nan, stats=['majority'], categorical=True)
majority_list = [{k: v for k, v in d.items() if k == 'majority'} for d in stats_siarl]
grid = grid.join(pd.DataFrame(majority_list), how='left')
grid = grid.rename(columns={"majority": "siarl"})

In [15]:
# Class counts in each tile
stats_siarl = rstat.zonal_stats(grid, siarl_array, affine=affine, nodata=np.nan, stats=['count'], categorical=True)
p = pd.DataFrame.from_dict(stats_siarl, orient='columns')
p = p*m2_to_km2

grid['siarl2'] = p[2.0]
grid['siarl7'] = p[7.0]
grid['siarl9'] = p[9.0]
grid['siarl12'] = p[12.0]

grid['siarlSum'] = p['count']

- - -

### [Digital Terrain Model - Geoportale Lombardia](https://www.geoportale.regione.lombardia.it/metadati?p_p_id=detailSheetMetadata_WAR_gptmetadataportlet&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&_detailSheetMetadata_WAR_gptmetadataportlet_uuid=%7BFC06681A-2403-481F-B6FE-5F952DD48BAF%7D)<br>
Digital Terrain Model of Lombardy region with 20 m resolution. Reference system EPSG:4326.
1. **Elevation** = DTM with 20 m resolution
2. **Aspect** = calculated previously from the elevation layer
3. **Slope** = calculated previously from the elevation layer

In [16]:
dtm_path = cwd + '/terrain/dtm20.tif'
aspect_path = cwd + '/terrain/aspect.tif'
slope_path = cwd + '/terrain/slope.tif'
dtm = rio.open(dtm_path)
aspect = rio.open(aspect_path)
slope = rio.open(slope_path)

In [17]:
#Calculates mean value in each cell for the elevation
dtm_array = dtm.read(1)
dtm_array[dtm_array<0]=np.nan
affine = dtm.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, dtm_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "h_mean"})

In [18]:
#Calculates mean value in each cell for aspect
aspect_array = aspect.read(1)
aspect_array[aspect_array<0]=np.nan
affine = aspect.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, aspect_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "aspect_mean"})

In [19]:
#Calculates mean value in each cell for slope
slope_array = slope.read(1)
slope_array[slope_array<0]=np.nan
affine = slope.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, slope_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "slope_mean"})

---

### Soil and Vegetation

In this part information concerning soil and vegetation is considered, such as:
- Soil type: classification fron OpenLandMap soil classification
- Soil text: classification obtained from (Carta Pedologica 1:250.000) from Geoportale Regione Lombardia
- Soil moist: soil moisture in soil layer 1 (0 - 7 cm) of the ECMWF Integrated Forecasting System
- NDVI: NDVI obtained from Terra Vegetation Indices 16-Day Global 250m dataset

USDA Soil Classification (missing value 3, 10, 12 in this Lombardy region): 
- 1 Clay
- 2 Silty Clay
- 3 Sandy Clay
- 4 Clay Loeam
- 5 Silty Clay Loam
- 6 Sandy Clay Loam
- 7 Loam
- 8 Silt Loam
- 9 Sandy Loam
- 10 Silt
- 11 Loamy Sand
- 12 Sand

In [20]:
soil_type_path = cwd + '/temp/soil_type.tif'
soil_moist_path = cwd + '/temp/soil_hum.tif'
ndvi_path = cwd + '/temp/ndvi.tif'
soil_text_path = cwd + '/temp/carta_pedologica_4326_dissolved_250K.tif'

soil_type = rio.open(soil_type_path)
soil_moist = rio.open(soil_moist_path)
ndvi = rio.open(ndvi_path)
soil_text = rio.open(soil_text_path)

In [21]:
soil_type_array = soil_type.read(1).astype('float64') 
soil_type_array[soil_type_array<1.0]=np.nan
affine = soil_type.transform

In [22]:
stats_soil_type = rstat.zonal_stats(grid, soil_type_array, affine=affine, nodata=np.nan, stats=['majority'], categorical=True)
majority_list = [{k: v for k, v in d.items() if k == 'majority'} for d in stats_soil_type]
grid = grid.join(pd.DataFrame(majority_list), how='left')
grid = grid.rename(columns={"majority": "soil"})

In [23]:
# Class counts in each tile
stats_soil = rstat.zonal_stats(grid, soil_type_array, affine=affine, nodata=np.nan, stats=['count'], categorical=True)
p = pd.DataFrame.from_dict(stats_soil, orient='columns')
grid['soil1'] = p[1.0]
grid['soil2'] = p[2.0]
#grid['soil3'] = p[3.0] #not present in lombardy region .tif
grid['soil4'] = p[4.0]
grid['soil5'] = p[5.0]
grid['soil6'] = p[6.0]
grid['soil7'] = p[7.0]
grid['soil8'] = p[8.0]
grid['soil9'] = p[9.0]
#grid['soil10'] = p[10.0] #not present in lombardy region .tif
grid['soil11'] = p[11.0]
#grid['soil12'] = p[12.0] #not present in lombardy region .tif

grid['soilSum'] = p['count']

Soil texture type:
- 1 Argillosa fine
- 2 Argillosa molto fine
- 3 Franca fine
- 4 Franca grossolana
- 5 Limosa fine
- 6 Limosa grossolana
- 7 Sabbiosa
- 8 Scheletrico-Argillosa
- 9 Scheletrico-Franca
- 10 Scheletrico-Sabbiosa

In [24]:
soil_text_array = soil_text.read(1).astype('float64') 
soil_text_array[soil_text_array<1.0]=np.nan
affine = soil_text.transform

In [26]:
stats_soil_text = rstat.zonal_stats(grid, soil_text_array, affine=affine, nodata=np.nan, stats=['majority'], categorical=True)
majority_list = [{k: v for k, v in d.items() if k == 'majority'} for d in stats_soil_text]
grid = grid.join(pd.DataFrame(majority_list), how='left')
grid = grid.rename(columns={"majority": "soil_text"})

In [27]:
# Class counts in each tile
stats_soil_text = rstat.zonal_stats(grid, soil_text_array, affine=affine, nodata=np.nan, stats=['count'], categorical=True)
p = pd.DataFrame.from_dict(stats_soil_text, orient='columns')
grid['soil_text1'] = p[1.0]
grid['soil_text2'] = p[2.0]
grid['soil_text3'] = p[3.0]
grid['soil_text4'] = p[4.0]
grid['soil_text5'] = p[5.0]
grid['soil_text6'] = p[6.0]
grid['soil_text7'] = p[7.0]
grid['soil_text8'] = p[8.0]
grid['soil_text9'] = p[9.0]
grid['soil_text10'] = p[10.0]

grid['soilTextSum'] = p['count']

Soil Moisture from ERA5 model:

In [28]:
soil_moist_array = soil_moist.read(1)
affine = soil_moist.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, soil_moist_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "soil_moist"})

NDVI from MODIS 16-days global:

In [29]:
ndvi_array = ndvi.read(1)
affine = ndvi.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, ndvi_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "ndvi"})

---

### Import climate:

In [30]:
# Set paths for temperature at 2m level, precipitation and ndvi
temp_2m_path = cwd + '/temp/temp_2m.tif'
prec_path = cwd + '/temp/prec.tif'
press_path = cwd + '/temp/press.tif'
n_wind_path = cwd + '/temp/n_wind.tif'
e_wind_path = cwd + '/temp/e_wind.tif'

temp_2m = rio.open(temp_2m_path)
prec = rio.open(prec_path)
press = rio.open(press_path)
n_wind = rio.open(n_wind_path)
e_wind = rio.open(e_wind_path)

In [31]:
temp_2m_array = temp_2m.read(1)
affine = temp_2m.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, temp_2m_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "temp_2m"})

In [32]:
prec_array = prec.read(1)
prec_array[prec_array<0]=np.nan
affine = prec.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, prec_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "prec"})

In [33]:
press_array = press.read(1)
press_array[press_array<0]=np.nan
affine = press.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, press_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "press"})

In [34]:
n_wind_array = n_wind.read(1)
affine = n_wind.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, n_wind_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "n_wind"})

In [35]:
e_wind_array = e_wind.read(1)
affine = e_wind.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, e_wind_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "e_wind"})

### Import pollutants

In [36]:
#From S5P
no2_path = cwd + '/temp/no2.tif'
so2_path = cwd + '/temp/so2.tif'
aod55_path = cwd + '/temp/aod_055.tif'
aod47_path = cwd + '/temp/aod_047.tif'
uvai_path = cwd + '/temp/uvai.tif'
co_path = cwd + '/temp/co.tif'
form_path = cwd + '/temp/form.tif'
ozone_path = cwd + '/temp/ozone.tif'

no2 = rio.open(no2_path)
so2 = rio.open(so2_path)
aod55 = rio.open(aod55_path)
aod47 = rio.open(aod47_path)
uvai = rio.open(uvai_path)
co = rio.open(co_path)
form = rio.open(form_path)
ozone = rio.open(ozone_path)

#From CAMS
amm_cams_path = cwd + '/temp/amm_cams.nc'
no_cams_path = cwd + '/temp/no_cams.nc'
co_cams_path = cwd + '/temp/co_cams.nc'
no2_cams_path = cwd + '/temp/no2_cams.nc'
dust_cams_path = cwd + '/temp/dust_cams.nc'
pm10_cams_path = cwd + '/temp/pm10_cams.nc'
pm25_cams_path = cwd + '/temp/pm25_cams.nc'
nmvocs_cams_path = cwd + '/temp/nmvocs_cams.nc'
so2_cams_path = cwd + '/temp/so2_cams.nc'
ozone_cams_path = cwd + '/temp/ozone_cams.nc'

amm_cams = rioxarray.open_rasterio(amm_cams_path,masked=True).rio.write_crs("epsg:4326", inplace=True)
no_cams = rioxarray.open_rasterio(no_cams_path,masked=True).rio.write_crs("epsg:4326", inplace=True)
co_cams = rioxarray.open_rasterio(co_cams_path,masked=True).rio.write_crs("epsg:4326", inplace=True)
no2_cams = rioxarray.open_rasterio(no2_cams_path,masked=True).rio.write_crs("epsg:4326", inplace=True)
dust_cams = rioxarray.open_rasterio(dust_cams_path,masked=True).rio.write_crs("epsg:4326", inplace=True)
pm10_cams = rioxarray.open_rasterio(pm10_cams_path,masked=True).rio.write_crs("epsg:4326", inplace=True)
pm25_cams = rioxarray.open_rasterio(pm25_cams_path,masked=True).rio.write_crs("epsg:4326", inplace=True)
nmvocs_cams = rioxarray.open_rasterio(nmvocs_cams_path,masked=True).rio.write_crs("epsg:4326", inplace=True)
so2_cams = rioxarray.open_rasterio(so2_cams_path,masked=True).rio.write_crs("epsg:4326", inplace=True)
ozone_cams = rioxarray.open_rasterio(ozone_cams_path,masked=True).rio.write_crs("epsg:4326", inplace=True)

#Convert ammonia to .tif
amm_cams.rio.to_raster(cwd + "/temp/amm_cams.tif")
amm_cams_path = cwd + '/temp/amm_cams.tif'
amm_cams = rio.open(amm_cams_path)
#Convert NO to .tif
no_cams.rio.to_raster(cwd + "/temp/no_cams.tif")
no_cams_path = cwd + '/temp/no_cams.tif'
no_cams = rio.open(no_cams_path)
#Convert CO to .tif
co_cams.rio.to_raster(cwd + "/temp/co_cams.tif")
co_cams_path = cwd + '/temp/co_cams.tif'
co_cams = rio.open(co_cams_path)
#Convert dust to .tif
dust_cams.rio.to_raster(cwd + "/temp/dust_cams.tif")
dust_cams_path = cwd + '/temp/dust_cams.tif'
dust_cams = rio.open(dust_cams_path)
#Convert pm10 to .tif
pm10_cams.rio.to_raster(cwd + "/temp/pm10_cams.tif")
pm10_cams_path = cwd + '/temp/pm10_cams.tif'
pm10_cams = rio.open(pm10_cams_path)
#Convert pm25 to .tif
pm25_cams.rio.to_raster(cwd + "/temp/pm25_cams.tif")
pm25_cams_path = cwd + '/temp/pm25_cams.tif'
pm25_cams = rio.open(pm25_cams_path)
#Convert NO2 to .tif
no2_cams.rio.to_raster(cwd + "/temp/no2_cams.tif")
no2_cams_path = cwd + '/temp/no2_cams.tif'
no2_cams = rio.open(no2_cams_path)
#Convert NMVOCs to .tif
nmvocs_cams.rio.to_raster(cwd + "/temp/nmvocs_cams.tif")
nmvocs_cams_path = cwd + '/temp/nmvocs_cams.tif'
nmvocs_cams = rio.open(nmvocs_cams_path)
#Convert SO2 to .tif
so2_cams.rio.to_raster(cwd + "/temp/so2_cams.tif")
so2_cams_path = cwd + '/temp/so2_cams.tif'
so2_cams = rio.open(so2_cams_path)
#Convert Ozone to .tif
ozone_cams.rio.to_raster(cwd + "/temp/ozone_cams.tif")
ozone_cams_path = cwd + '/temp/ozone_cams.tif'
ozone_cams = rio.open(ozone_cams_path)

### Sentinel-5P Data

In [37]:
no2_array = no2.read(1)
affine = no2.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, no2_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "no2_s5p"})

In [38]:
so2_array = so2.read(1)
affine = so2.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, so2_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "so2_s5p"})

In [39]:
aod55_array = aod55.read(1)
affine = aod55.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, aod55_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "aod_055"})

In [40]:
aod47_array = aod47.read(1)
affine = aod47.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, aod47_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "aod_047"})

In [41]:
uvai_array = uvai.read(1)
affine = uvai.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, uvai_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "uvai"})

In [42]:
co_array = co.read(1)
affine = co.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, co_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "co_s5p"})

In [43]:
form_array = form.read(1)
affine = form.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, form_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "form_s5p"})

In [44]:
ozone_array = ozone.read(1)
affine = ozone.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, ozone_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "ozone_s5p"})

### CAMS Data

Resample CAMS data (original resolution 0.1°): zonal stats has issues with original resolution

In [45]:
for files in os.listdir(cwd+'/temp'):
    if files[-9:] == '_cams.tif':
        file_name = files[:(len(files)-9)]
        xds = rioxarray.open_rasterio(cwd + "/temp/"+files,masked=True)
        new_width = xds.rio.width * upscale_factor
        new_height = xds.rio.height * upscale_factor
        xds_upsampled = xds.rio.reproject(xds.rio.crs,shape=(new_height, new_width),resampling=Resampling.bilinear)
        xds_upsampled.rio.to_raster(cwd +'/temp/'+file_name+'_cams_upsampled.tif')

In [46]:
amm_cams_path = cwd + '/temp/amm_cams_upsampled.tif'
amm_cams = rio.open(amm_cams_path)

no_cams_path = cwd + '/temp/no_cams_upsampled.tif'
no_cams = rio.open(no_cams_path)

co_cams_path = cwd + '/temp/co_cams_upsampled.tif'
co_cams = rio.open(co_cams_path)

dust_cams_path = cwd + '/temp/dust_cams_upsampled.tif'
dust_cams = rio.open(dust_cams_path)

pm10_cams_path = cwd + '/temp/pm10_cams_upsampled.tif'
pm10_cams = rio.open(pm10_cams_path)

pm25_cams_path = cwd + '/temp/pm25_cams_upsampled.tif'
pm25_cams = rio.open(pm25_cams_path)

no2_cams_path = cwd + '/temp/no2_cams_upsampled.tif'
no2_cams = rio.open(no2_cams_path)

nmvocs_cams_path = cwd + '/temp/nmvocs_cams_upsampled.tif'
nmvocs_cams = rio.open(nmvocs_cams_path)

so2_cams_path = cwd + '/temp/so2_cams_upsampled.tif'
so2_cams = rio.open(so2_cams_path)

ozone_cams_path = cwd + '/temp/ozone_cams_upsampled.tif'
ozone_cams = rio.open(ozone_cams_path)

In [47]:
amm_cams_array = amm_cams.read(1)
affine = amm_cams.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, amm_cams_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "amm_cams"})

In [48]:
no_cams_array = no_cams.read(1)
affine = no_cams.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, no_cams_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "no_cams"})

In [49]:
co_cams_array = co_cams.read(1)
affine = co_cams.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, co_cams_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "co_cams"})

In [50]:
dust_cams_array = dust_cams.read(1)
affine = dust_cams.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, dust_cams_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "dust_cams"})

In [51]:
pm10_cams_array = pm10_cams.read(1)
affine = pm10_cams.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, pm10_cams_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "pm10_cams"})

In [52]:
pm25_cams_array = pm25_cams.read(1)
affine = pm25_cams.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, pm25_cams_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "pm25_cams"})

In [53]:
no2_cams_array = no2_cams.read(1)
affine = no2_cams.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, no2_cams_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "no2_cams"})

In [54]:
nmvocs_cams_array = nmvocs_cams.read(1)
affine = nmvocs_cams.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, nmvocs_cams_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "nmvocs_cams"})

In [55]:
so2_cams_array = so2_cams.read(1)
affine = so2_cams.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, so2_cams_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "so2_cams"})

In [56]:
ozone_cams_array = ozone_cams.read(1)
affine = ozone_cams.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, ozone_cams_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "ozone_cams"})

### [Gridded Population of the World - GPW](https://sedac.ciesin.columbia.edu/data/set/gpw-v4-population-density-rev11)<br>
To provide estimates of population density for the year 2020, based on counts consistent with national censuses and population registers, as raster data to facilitate data integration.
Input reference system EPSG: 4326

In [57]:
pop_path = cwd + '/population/population.tif'
pop = rio.open(pop_path)

In [58]:
pop_array = pop.read(1)
pop_array[pop_array<0]=np.nan
affine = pop.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, pop_array, affine=affine, nodata=np.nan, stats=['sum'])), how='left')
grid = grid.rename(columns={"sum": "pop"})

 - - -

### [Road Infrastructures - Geoportale Lombardia (DBTR 2019)](https://www.geoportale.regione.lombardia.it/metadati?p_p_id=detailSheetMetadata_WAR_gptmetadataportlet&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&_detailSheetMetadata_WAR_gptmetadataportlet_uuid=%7B17D4656F-2E9D-4951-9DC1-4AD32C0959B1%7D): 

**Point layers** considered:
1. Intersection between primary roads including highways
2. Intersection between primary and secondary roads
3. Intersection between secondary roads

Input reference system EPSG: 4326

In [59]:
int_prim_path = cwd + '/road_infrastructures/inters_highway_prim_road.gpkg'
int_prim_sec_path = cwd + '/road_infrastructures/inters_prim_sec_road.gpkg'
int_sec_path = cwd + '/road_infrastructures/inters_sec_road.gpkg'

grid = grid.to_crs(32632)

In [60]:
int_prim = gpd.read_file(int_prim_path).to_crs(32632)
int_prim_sec = gpd.read_file(int_prim_sec_path).to_crs(32632)
int_sec = gpd.read_file(int_sec_path).to_crs(32632)

df_dict = {'int_prim':int_prim,
          'int_prim_sec':int_prim_sec, 'int_sec': int_sec}


for key in df_dict:
    poor_points = df_dict[key][['OBJECTID','geometry']]
    sjoined = gpd.sjoin(poor_points, grid)
    df_count = pd.DataFrame(sjoined.groupby('index_right').size()) 
    grid_join = grid.join(df_count)
    grid[key] = grid_join[0]

**Line layers** considered:
1. Highways
2. Primary roads
3. Secondary roads

Input reference system EPSG: 4326

In [61]:
highway_path = cwd + '/road_infrastructures/highway.gpkg'
prim_road_path = cwd + '/road_infrastructures/prim_road.gpkg'
sec_road_path = cwd + '/road_infrastructures/sec_road.gpkg'

It is required to convert to a cartographic reference system EPSG:32632 to calculate distances.

In [62]:
highway = gpd.read_file(highway_path).to_crs(32632)
prim_road = gpd.read_file(prim_road_path).to_crs(32632)
sec_road = gpd.read_file(sec_road_path).to_crs(32632)

In [63]:
df_dict = {'highway':highway, 'prim_road':prim_road, 'sec_road':sec_road}

for key in df_dict:
    grid[key] = np.nan
    poor_lines = df_dict[key][['geodb_oid','geometry']]
    for index, row in grid.iterrows():
        mask = row['geometry']
        clip = gpd.clip(poor_lines, mask) 
        l = clip.geometry.length.sum()
        grid[key].iloc[index] = l*m_to_km
    print(key)

highway
prim_road
sec_road


 - - -

### Farms
Vector file obtained from DUSAF 2018 (features with cod. 12112 = "Insediamenti produttivi agricoli
Sono compresi in questa classe gli edifici utilizzati per le attività produttive del settore primario, come capannoni, rimesse per macchine agricole, fienili, stalle, silos, ecc, unitamente agli spazi accessori. Quando tali edifici sono presenti insieme a quelli residenziali configurando un aggregato rurale, se le due tipologie non risultano separabili in modo evidente si classifica tutto il nucleo come cascina (11231)").

In [64]:
farms_path = cwd + '/farms/farms_dissolve.gpkg'
farms = gpd.read_file(farms_path)

In [65]:
df_dict2 = {'farms':farms}

In [66]:
for key in df_dict2:
    grid[key] = np.nan
    poor_poly = df_dict2[key][['COD_TOT','geometry']]
    for index, row in grid.iterrows():
        mask = row['geometry']
        clip = gpd.clip(poor_poly, mask) 
        a = clip.geometry.area.sum()
        grid[key].iloc[index] = a*m2_to_km2
    print(key)

farms


In [67]:
grid = grid.to_crs(4326)

### Breeding farm type

In [68]:
pigs_path = cwd + '/farms/pigs.gpkg'
poultry_path = cwd + '/farms/poultry.gpkg'
sheep_path = cwd + '/farms/sheep.gpkg'

In [69]:
farm_pigs = gpd.read_file(pigs_path)
farm_poultry = gpd.read_file(poultry_path)
farm_sheep = gpd.read_file(sheep_path)

df_dict = {'farm_pigs':farm_pigs,
          'farm_poultry':farm_poultry, 'farm_sheep':farm_sheep}


for key in df_dict:
    poor_points = df_dict[key][['OBJECTID','geometry']]
    sjoined = gpd.sjoin(poor_points, grid)
    df_count = pd.DataFrame(sjoined.groupby('index_right').size()) 
    grid_join = grid.join(df_count)
    grid[key] = grid_join[0]
    print(key)

farm_pigs
farm_poultry
farm_sheep


---

# Import air quality and meteo stations 

Assign to each cell the value measured by each sensor. If more sensors of the same type are present in the same cell it assign the mean value:

### Meteo stations:

In [70]:
temp_st_path = cwd + '/temp/temp_st.gpkg'
prec_st_path = cwd + '/temp/prec_st.gpkg'
air_hum_st_path = cwd + '/temp/air_hum_st.gpkg'
wind_dir_st_path = cwd + '/temp/wind_dir_st.gpkg'
wind_speed_st_path = cwd + '/temp/wind_speed_st.gpkg'
rad_glob_st_path = cwd + '/temp/rad_glob_st.gpkg'

In [71]:
temp_st = gpd.read_file(temp_st_path)
prec_st = gpd.read_file(prec_st_path)
air_hum_st = gpd.read_file(air_hum_st_path)
wind_dir_st = gpd.read_file(wind_dir_st_path)
wind_speed_st = gpd.read_file(wind_speed_st_path)
rad_glob_st = gpd.read_file(rad_glob_st_path)

In [72]:
df_dict = {'temp_st':temp_st,
          'prec_st':prec_st, 'air_hum_st': air_hum_st, 'wind_dir_st':wind_dir_st, 'wind_speed_st':wind_speed_st,
          'rad_glob_st':rad_glob_st}

In [73]:
for key in df_dict:
    grid[key] = np.nan
    poor_points = df_dict[key][['idsensore','valore','geometry']]
    for index, row in grid.iterrows():
        mask = row['geometry']
        clip = gpd.clip(poor_points, mask) 
        m = clip.valore.mean()
        grid[key].iloc[index] = m
    print(key)

temp_st
prec_st
air_hum_st
wind_dir_st
wind_speed_st
rad_glob_st


### Air quality station:

In [74]:
pm25_st_path = cwd + '/temp/pm25_st.gpkg'
nox_st_path = cwd + '/temp/nox_st.gpkg'
no2_st_path = cwd + '/temp/no2_st.gpkg'
amm_st_path = cwd + '/temp/amm_st.gpkg'
so2_st_path = cwd + '/temp/so2_st.gpkg'
pm10_st_path = cwd + '/temp/pm10_st.gpkg'
co_st_path = cwd + '/temp/co_st.gpkg'
ozone_st_path = cwd + '/temp/o3_st.gpkg'

In [75]:
pm25_st = gpd.read_file(pm25_st_path)
nox_st = gpd.read_file(nox_st_path)
no2_st = gpd.read_file(no2_st_path)
amm_st = gpd.read_file(amm_st_path)
so2_st = gpd.read_file(so2_st_path)
pm10_st = gpd.read_file(pm10_st_path)
co_st = gpd.read_file(co_st_path)
ozone_st = gpd.read_file(ozone_st_path)

In [76]:
df_dict = {'pm25_st':pm25_st,
          'nox_st':nox_st, 'no2_st': no2_st,'ozone_st':ozone_st, 'amm_st':amm_st, 'so2_st':so2_st, 'pm10_st':pm10_st, 'co_st':co_st}

In [77]:
for key in df_dict:
    grid[key] = 0
    poor_points = df_dict[key][['idsensore','valore','geometry']]
    for index, row in grid.iterrows():
        mask = row['geometry']
        clip = gpd.clip(poor_points, mask) 
        m = clip.valore.mean()
        grid[key].iloc[index] = m
    print(key)

pm25_st
nox_st
no2_st
ozone_st
amm_st
so2_st
pm10_st
co_st


### Import Low Cost Sensor data ([ESA Air Quality Sensors](https://aqp.eo.esa.int/map/))

In [78]:
pm25_lcs_path = cwd + '/temp/pm25_lcs.gpkg'
pm10_lcs_path = cwd + '/temp/pm10_lcs.gpkg'
hum_lcs_path = cwd + '/temp/hum_lcs.gpkg'
temp_lcs_path = cwd + '/temp/temp_lcs.gpkg'
no2_lcs_path = cwd + '/temp/no2_lcs.gpkg'
co2_lcs_path = cwd + '/temp/co2_lcs.gpkg'
amm_lcs_path = cwd + '/temp/amm_lcs.gpkg'
co_lcs_path = cwd + '/temp/co_lcs.gpkg'

In [79]:
pm25_lcs = gpd.read_file(pm25_lcs_path)
pm10_lcs = gpd.read_file(pm10_lcs_path)
hum_lcs = gpd.read_file(hum_lcs_path)
temp_lcs = gpd.read_file(temp_lcs_path)
no2_lcs = gpd.read_file(no2_lcs_path)
co2_lcs = gpd.read_file(co2_lcs_path)
amm_lcs = gpd.read_file(amm_lcs_path)
co_lcs = gpd.read_file(co_lcs_path)

In [80]:
df_dict = {'pm25_lcs':pm25_lcs,
          'pm10_lcs':pm10_lcs, 'hum_lcs': hum_lcs, 'temp_lcs':temp_lcs, 'no2_lcs':no2_lcs,
          'co2_lcs':co2_lcs, 'amm_lcs':amm_lcs, 'co_lcs':co_lcs}

In [81]:
for key in df_dict:
    grid[key] = 0
    poor_points = df_dict[key][['device_id','value','geometry']]
    for index, row in grid.iterrows():
        mask = row['geometry']
        clip = gpd.clip(poor_points, mask) 
        m = clip.value.mean()
        grid[key].iloc[index] = m
    print(key)

pm25_lcs
pm10_lcs
hum_lcs
temp_lcs
no2_lcs
co2_lcs
amm_lcs
co_lcs


### Import air quality and meteorological data from station interpolated data

Export the file in WGS84:

In [82]:
grid.to_crs(4326).to_file(cwd+"/results/test_farm_grid.gpkg", driver="GPKG")

In [98]:
corr = grid.corr()

In [100]:
corr.to_csv('prova.csv')

### Interpolation

Use a .tif raster (for example aod_055.tif) file to get the parameters for the interpolated raster:

In [83]:
# aod = gdal.Open(cwd+"/temp/aod_055.tif")
# gt = aod.GetGeoTransform()

In [84]:
# ulx = gt[0]
# uly = gt[3]
# res = gt[1]

In [85]:
# xsize = aod.RasterXSize
# ysize = aod.RasterYSize

In [86]:
# lrx = ulx + xsize * res
# lry = uly - ysize * res

In [87]:
# rRes = 0.01

In [88]:
# lon= air_hum_st['lng']
# lat =air_hum_st['lat']
# value = air_hum_st['valore']

In [89]:
# xRange = np.arange(ulx,lrx+rRes,rRes)
# yRange = np.arange(lry,uly+rRes,rRes)
# gridX,gridY = np.meshgrid(xRange, yRange)

In [90]:
# OK = OrdinaryKriging(lon, lat, value,
#                      variogram_model='gaussian', verbose=True, enable_plotting=False,nlags=10)

In [91]:
# print(OrdinaryKriging.__doc__ )

In [92]:
# gridded, ss1 = OK.execute('grid', xRange, yRange)

In [93]:
# plt.imshow(gridded)

In [94]:
# from rasterio.transform import Affine
# transform = Affine.translation(gridX[0][0]-rRes/2, gridY[0][0]-rRes/2)*Affine.scale(rRes,rRes)

In [95]:
# from rasterio.crs import CRS
# rasterCrs = CRS.from_epsg(4326)

In [96]:
# #definition, register and close of interpolated raster
# interpRaster = rio.open(cwd+'/kriging.tif',
#                                 'w',
#                                 driver='GTiff',
#                                 height=gridded.shape[0],
#                                 width=gridded.shape[1],
#                                 count=1,
#                                 nodata = -9999,
#                                 dtype=gridded.dtype,
#                                 crs=rasterCrs,
#                                 transform=transform,
#                                 )
# interpRaster.write(gridded, 1)
# interpRaster.close()

In [97]:
# pts = ogr.Open(cwd+'/temp/air_hum_st.gpkg', update = True)
# layer=pts.GetLayer()

# to generate a an interpolation using GDAL library
# pts = layer = None
# idw = gdal.Grid("idw.tif", (cwd+'/temp/air_hum_st.gpkg'), zfield="valore",
#                algorithm = "invdist", outputBounds = [ulx,uly,lrx,lry],
#                width = xsize, height = ysize)
# idw = None

# points = list(zip(air_hum_st.lng,air_hum_st.lat))

# gridded = griddata(points, value, (gridX,gridY), method='linear',fill_value=0)

# gridded = gridded.reshape((gridded.shape[0], gridded.shape[1]))

# plt.imshow(gridded)